
<h1 id="1.-데이터-로드">1. 데이터 로드<a class="anchor-link" href="#1.-데이터-로드">¶</a></h1>


In [ ]:

!pip install PyDrive



In [ ]:

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials



In [ ]:

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [ ]:

#구글드라이브에서 파일 우클릭 > 공유가능한 링크가져오기 > 복붙한다음 id뒤에만 복붙하기
# https://drive.google.com/open?id=1F67mZanxKNSI8RCmgPAFcTlMoPZS7ulX



In [ ]:

# 내 드라이브의 파일
# https://drive.google.com/open?id=13Fouf0a0o57kLQjpVk23DmZJl08CFHhH



In [ ]:

download = drive.CreateFile({"id": "13Fouf0a0o57kLQjpVk23DmZJl08CFHhH"})
download.GetContentFile("wine2.csv")



In [ ]:

import pandas as pd
import numpy as np



In [ ]:

df = pd.read_csv("wine2.csv")



In [ ]:

df




<h1 id="2.-데이터-전처리">2. 데이터 전처리<a class="anchor-link" href="#2.-데이터-전처리">¶</a></h1>



<h3 id="1)데이터-요약">1)데이터 요약<a class="anchor-link" href="#1)데이터-요약">¶</a></h3>


In [ ]:

df.describe()



In [ ]:

df.shape



In [ ]:

#범주형 변수가 없다
df.info()



In [ ]:

#다행히 데이터에 결측치가 없다
df.isnull().sum()




<h3 id="2)라벨-확인">2)라벨 확인<a class="anchor-link" href="#2)라벨-확인">¶</a></h3>


In [ ]:

#imbalanced 라벨이다
from collections import Counter
Counter(df["Wine"])



In [ ]:

#기존의 라벨을 시각화해보자
from matplotlib import pyplot as plt
import matplotlib



In [ ]:

fig = plt.figure(figsize = (8,5))
colors = ['red', 'green', 'blue']
y=df["Wine"]
plt.scatter(df["Alcohol"], df["Malic.acid"], c=y, cmap = matplotlib.colors.ListedColormap(colors))
plt.show()




<h3 id="3)피쳐-분석">3)피쳐 분석<a class="anchor-link" href="#3)피쳐-분석">¶</a></h3>


In [ ]:

import seaborn as sns



In [ ]:

#피쳐간 그래프
features = df.drop(["Wine"], axis=1)
sns.pairplot(features)
#중간에 관계가 강한 피쳐들이 보인다



In [ ]:

#피쳐의 상관계수 시각화
plt.figure(figsize=(15,15))
sns.heatmap(data = df.drop(["Wine"], axis=1).corr(), annot=True, fmt='.2f', linewidths=.5, cmap="Blues")



In [ ]:

#VIF로 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(
    features.values, i) for i in range(features.shape[1])]



In [ ]:

#VIF는 다른 피쳐로 해당 피쳐를 선형회귀한 성능을 나타내는 지표로, VIF값이 클수록 다른 변수에 의존성이 크다
vif["features"] = features.columns
vif



In [ ]:

#VIF값이 큰 순으로 정렬
vif.sort_values(by=['VIF Factor'], ascending=False)



In [ ]:

#Alcohol과 Ash의 VIF값이 상당히 크므로, 이후 기계학습시에 성능이 낮으면 해당 피쳐를 삭제해보는 것을 고려할 수 있음!
#다중공선성이 큰 두개의 값을 삭제한 새로운 피쳐

new_features = features.drop(["Alcohol", "Ash"], axis=1)




<ul>
<li>로지스틱 회귀는 다중공선성이 높은 피쳐를 제거한 데이터로 학습해야 성능이 좋았지만, SVM은 비슷했고 DT와 Ensemble은 원래 데이터로 학습했을 때 성능이 훨씬 좋았습니다. </li>
<li>그래서 아래 데이터 분석에서는 기존의 피쳐를 사용했습니다.</li>
</ul>



<h1 id="3.-데이터-분석">3. 데이터 분석<a class="anchor-link" href="#3.-데이터-분석">¶</a></h1>



<h3 id="1)train_test_split">1)train_test_split<a class="anchor-link" href="#1)train_test_split">¶</a></h3>


In [ ]:

X = features
y = df["Wine"]



In [ ]:

#train data와 test data를 stratified resampling(층화추출)
from sklearn.model_selection import train_test_split

X_dev, X_test, y_dev, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42, stratify = y)




<h3 id="2)Oversampling전-데이터로-Modeling">2)Oversampling전 데이터로 Modeling<a class="anchor-link" href="#2)Oversampling전-데이터로-Modeling">¶</a></h3>



<ul>
<li>a. Logistic Regression</li>
</ul>


In [ ]:

#Logistic modeling
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)



In [ ]:

#grid search
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], "penalty":["l1","l2"]} # l1 lasso l2 ridge
lr_gs = GridSearchCV(lr, param_grid, cv=5)
lr_gs = lr_gs.fit(X_dev, y_dev)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(lr_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
lr = LogisticRegression(random_state=42, C=1000, penalty='l1').fit(X_dev, y_dev)



In [ ]:

#test set으로 검증
from sklearn.metrics import accuracy_score, classification_report

y_pred = lr.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
lr_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<ul>
<li>b. SVM</li>
</ul>


In [ ]:

#SVM modeling
from sklearn import svm

svm_clf = svm.SVC(gamma="scale", random_state=42)



In [ ]:

#grid search
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100]}
svm_gs = GridSearchCV(svm_clf, parameters, cv=5)
svm_gs = svm_gs.fit(X_dev, y_dev)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(svm_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
svm = svm.SVC(gamma="scale", random_state=42, C=1, kernel='linear').fit(X_dev, y_dev)



In [ ]:

#test set으로 검증
y_pred = svm.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
svm_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])




<ul>
<li>c. DT</li>
</ul>


In [ ]:

#DT modeling
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)



In [ ]:

#DT grid search
param_grid = {'criterion':["gini", "entropy"], "min_samples_split":[2,4,6],
               "min_samples_leaf":[1,3,5]}
dt_gs = GridSearchCV(dt, param_grid, cv=5)
dt_gs = dt_gs.fit(X_dev, y_dev)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(dt_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
dt = DecisionTreeClassifier(random_state=42, criterion='entropy', min_samples_leaf=1, min_samples_split=6).fit(X_dev, y_dev)



In [ ]:

#test set으로 검증
y_pred = dt.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
dt_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<ul>
<li>d. Ensemble</li>
</ul>


In [ ]:

#Ensemble modeling - bagging 중 random forest
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)



In [ ]:

#Ensemble grid search
param_grid = { 
    'n_estimators': [10, 30, 50, 70],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8],
    'criterion' :['gini', 'entropy']
}
rf_gs = GridSearchCV(rf, param_grid, cv=5)
rf_gs = rf_gs.fit(X_dev, y_dev)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(rf_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
rf = RandomForestClassifier(random_state=42, criterion='gini',max_depth=4, max_features='auto', n_estimators=30).fit(X_dev, y_dev)



In [ ]:

#test set으로 검증
y_pred = rf.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
rf_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<h3 id="3)Random-oversampling-후-Modeling">3)Random oversampling 후 Modeling<a class="anchor-link" href="#3)Random-oversampling-후-Modeling">¶</a></h3>


In [ ]:

from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_ros, y_ros = ros.fit_sample(X_dev, y_dev)




<ul>
<li>a. Logistic Regression</li>
</ul>


In [ ]:

#Logistic modeling
ros_lr = LogisticRegression(random_state=42)



In [ ]:

#grid search
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], "penalty":["l1","l2"]} # l1 lasso l2 ridge
ros_lr_gs = GridSearchCV(ros_lr, param_grid, cv=5)
ros_lr_gs = ros_lr_gs.fit(X_ros, y_ros)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(ros_lr_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
ros_lr = LogisticRegression(random_state=42, C=1000, penalty='l1').fit(X_ros, y_ros)



In [ ]:

#test set으로 검증
y_pred = ros_lr.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
ros_lr_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<ul>
<li>b. SVM</li>
</ul>


In [ ]:

#SVM modeling
from sklearn import svm

svm_clf = svm.SVC(gamma="scale", random_state=42)



In [ ]:

#grid search
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100]}
ros_svm_gs = GridSearchCV(svm_clf, parameters, cv=5)
ros_svm_gs = ros_svm_gs.fit(X_ros, y_ros)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(ros_svm_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
ros_svm = svm.SVC(gamma="scale", random_state=42, C=10, kernel='linear').fit(X_ros, y_ros)



In [ ]:

#test set으로 검증
y_pred = ros_svm.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
ros_svm_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])




<ul>
<li>c. DT</li>
</ul>


In [ ]:

#DT modeling
from sklearn.tree import DecisionTreeClassifier

ros_dt = DecisionTreeClassifier(random_state=42)



In [ ]:

#DT grid search
param_grid = {'criterion':["gini", "entropy"], "min_samples_split":[2,4,6],
               "min_samples_leaf":[1,3,5]}
ros_dt_gs = GridSearchCV(ros_dt, param_grid, cv=5)
ros_dt_gs = ros_dt_gs.fit(X_ros, y_ros)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(ros_dt_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
ros_dt = DecisionTreeClassifier(random_state=42, criterion='gini',
                                min_samples_leaf=1, min_samples_split=2).fit(X_ros, y_ros)



In [ ]:

#test set으로 검증
y_pred = ros_dt.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
ros_dt_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<ul>
<li>d. Ensemble</li>
</ul>


In [ ]:

#Ensemble modeling - bagging 중 random forest
from sklearn.ensemble import RandomForestClassifier

ros_rf = RandomForestClassifier(random_state=42)



In [ ]:

#Ensemble grid search
param_grid = { 
    'n_estimators': [10, 30, 50, 70],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8],
    'criterion' :['gini', 'entropy']
}
ros_rf_gs = GridSearchCV(ros_rf, param_grid, cv=5)
ros_rf_gs = ros_rf_gs.fit(X_ros, y_ros)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(ros_rf_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
ros_rf = RandomForestClassifier(random_state=42, criterion='gini',max_depth=4,
                            max_features='auto', n_estimators=30).fit(X_ros, y_ros)



In [ ]:

#test set으로 검증
y_pred = ros_rf.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
ros_rf_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<h3 id="3.-SMOTE-Oversampling-후-Modeling">3. SMOTE Oversampling 후 Modeling<a class="anchor-link" href="#3.-SMOTE-Oversampling-후-Modeling">¶</a></h3>


In [ ]:

from imblearn.over_sampling import SMOTE

smothe = SMOTE()
X_smt, y_smt = smothe.fit_sample(X_dev, y_dev)




<ul>
<li>a.Logistic Regression</li>
</ul>


In [ ]:

#Logistic modeling
smt_lr = LogisticRegression(random_state=42)



In [ ]:

#grid search
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], "penalty":["l1","l2"]} # l1 lasso l2 ridge
smt_lr_gs = GridSearchCV(smt_lr, param_grid, cv=5)
smt_lr_gs = smt_lr_gs.fit(X_smt, y_smt)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(smt_lr_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
smt_lr = LogisticRegression(random_state=42, C=1000, penalty='l1').fit(X_smt, y_smt)



In [ ]:

#test set으로 검증
y_pred = smt_lr.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
smt_lr_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])




<ul>
<li>b.SVM</li>
</ul>


In [ ]:

#SVM modeling
from sklearn import svm

svm_clf = svm.SVC(gamma="scale", random_state=42)



In [ ]:

#grid search
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100]}
smt_svm_gs = GridSearchCV(svm_clf, parameters, cv=5)
smt_svm_gs = smt_svm_gs.fit(X_smt, y_smt)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(smt_svm_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
smt_svm = svm.SVC(gamma="scale", random_state=42, C=1, kernel='linear').fit(X_smt, y_smt)



In [ ]:

#test set으로 검증
y_pred = smt_svm.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
smt_svm_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])




<ul>
<li>c.DT</li>
</ul>


In [ ]:

#DT modeling
from sklearn.tree import DecisionTreeClassifier

smt_dt = DecisionTreeClassifier(random_state=42)



In [ ]:

#DT grid search
param_grid = {'criterion':["gini", "entropy"], "min_samples_split":[2,4,6],
               "min_samples_leaf":[1,3,5]}
smt_dt_gs = GridSearchCV(smt_dt, param_grid, cv=5)
smt_dt_gs = smt_dt_gs.fit(X_smt, y_smt)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(smt_dt_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
smt_dt = DecisionTreeClassifier(random_state=42, criterion='gini',
                                min_samples_leaf=1, min_samples_split=2).fit(X_smt, y_smt)



In [ ]:

#test set으로 검증
y_pred = smt_dt.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
smt_dt_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<ul>
<li>d. Ensemble</li>
</ul>


In [ ]:

#Ensemble modeling - bagging 중 random forest
from sklearn.ensemble import RandomForestClassifier

smt_rf = RandomForestClassifier(random_state=42)



In [ ]:

#Ensemble grid search
param_grid = { 
    'n_estimators': [10, 30, 50, 70],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,6,8],
    'criterion' :['gini', 'entropy']
}
smt_rf_gs = GridSearchCV(smt_rf, param_grid, cv=5)
smt_rf_gs = smt_rf_gs.fit(X_smt, y_smt)



In [ ]:

#grid search결과
print("tuned hyperparameters : {}".format(smt_rf_gs.best_params_))



In [ ]:

#최적의 파라미터로 모델링
smt_rf = RandomForestClassifier(random_state=42, criterion='entropy',max_depth=6,
                            max_features='auto', n_estimators=70).fit(X_smt, y_smt)



In [ ]:

#test set으로 검증
y_pred = smt_rf.predict(X_test)
print("Accurancy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["1", "2", "3"]))
smt_rf_report = classification_report(y_test, y_pred, target_names=["1", "2", "3"])
#imbalance 라벨이므로, 정확도(accuracy)보다는 precision과 recall을 통해 모델 정확도를 평가하는게 좋음




<h1 id="4.-성능비교">4. 성능비교<a class="anchor-link" href="#4.-성능비교">¶</a></h1>



<ul>
<li>a. Logistic Regression</li>
</ul>


In [ ]:

#oversampling전 성능
print(lr_report)



In [ ]:

#Random oversampling후 성능
print(ros_lr_report)



In [ ]:

#SMOTE oversampling후 성능
print(smt_lr_report)




<p>-- Logistic Regression은 oversampling 전과 후 성능차이가 나타나지 않았다.</p>



<ul>
<li>b. SVM</li>
</ul>


In [ ]:

#oversampling전 성능
print(svm_report)



In [ ]:

#Random oversampling후 성능
print(ros_svm_report)



In [ ]:

#SMOTE oversampling후 성능
print(smt_svm_report)




<p>-- SVM의 oversampling 전과 SMOTE oversampling후 성능은 비슷하지만, Random oversampling은 성능이 다소 떨어졌다. 아마도 random으로 생성한 데이터라서 그런 것 같다.</p>



<ul>
<li>c. DT</li>
</ul>


In [ ]:

#oversampling전 성능
print(dt_report)



In [ ]:

#Random oversampling후 성능
print(ros_dt_report)



In [ ]:

#SMOTE oversampling후 성능
print(smt_dt_report)




<p>-- DT는 oversampling 전에 성능이 더 좋았다. 아래의 RandomForest도 비슷한 경향인 것으로 보아 트리계열 모델은 oversampling보다 원래 데이터로 학습하는게 성능을 높이는 방법인것같다.</p>



<p>-- 또한 다중공선성이 큰 피쳐를 삭제한 데이터로 트리계열 모델을 학습시켰을 때 원래보다 성능이 떨어졌다. 트리계열이 위에서부터 가장 의미있는 피쳐를 사용하기 때문에, 굳이 다중공선성이 큰 피쳐를 삭제하지 않는게 좋은 것같다.</p>



<ul>
<li>d. Ensemble</li>
</ul>


In [ ]:

#oversampling 전 성능
print(rf_report)



In [ ]:

#Random oversampling 후 성능
print(ros_rf_report)



In [ ]:

#SMOTE oversamping 후 성능
print(smt_rf_report)




<p>-- RandomForest는 oversampling전과 SMOTE oversampling시 성능이 가장 좋았고, random oversampling시 성능이 가중평균을 기준으로 0.03정도 감소하였다. random으로 데이터를 생성했기 때문인것 같다.</p>
